# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:
cities_df = pd.read_csv("weather_data.csv")
cities_df

,City,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,Moose Factory,51.260217,-80.603282,37.38,87,100,11.50
1,Yellowknife,62.454081,-114.377385,48.22,57,83,5.01
2,Hilo CDP,19.707373,-155.081580,80.28,74,40,6.91
3,Havøysund - Ávanuorri,70.996311,24.660958,44.83,93,100,41.14
4,Carnarvon,-24.882613,113.657649,68.00,37,0,21.45
...,...,...,...,...,...,...,...
584,Vestmanna,62.156000,-7.166000,45.64,87,75,3.44
585,Narayanganj,23.623673,90.498807,84.22,78,94,3.36
586,Hervey Bay,-25.298578,152.853522,77.02,58,84,7.00
587,Arcata,40.866517,-124.082840,57.15,88,100,3.47


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure(center = (10,0), zoom_level = 2)
heatmap_layer = gmaps.heatmap_layer(
    cities_df[['Latitude', 'Longitude']],
    weights=cities_df['Humidity'], 
    max_intensity = 100, point_radius = 8)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [45]:
narrow_df = cities_df[cities_df["Temperature (F)"] <= 80]
narrow_df = narrow_df[narrow_df["Temperature (F)"] > 70]
narrow_df = narrow_df[narrow_df["Wind Speed (mph)"] < 10]
narrow_df = narrow_df[narrow_df["Cloudiness"] == 0]
narrow_df.dropna()
narrow_df

,City,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
117,Choctaw,35.497565,-97.268926,70.02,47,0,5.75
208,Viranşehir,37.232895,39.762014,74.57,21,0,5.64
214,Ampanihy,-24.694126,44.747682,70.12,52,0,0.94
242,Hasilpur,29.688981,72.552545,76.80,71,0,9.13
275,Turbat,26.002789,63.050561,78.39,11,0,7.09
311,Tsihombe,-25.318547,45.484057,71.46,58,0,0.34
397,Cizre,37.332437,42.185470,72.57,24,0,9.22
467,Riyadh,24.638916,46.716010,73.78,15,0,5.48
505,Cayenne,4.937114,-52.325831,77.02,88,0,3.44
557,Sehithwa,-20.473311,22.701307,71.69,20,0,4.27


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
hotel_df = narrow_df

In [75]:
url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&"
count = 0
hotel_names = []
addresses = []
for city in hotel_df["City"]:
    lat = hotel_df.iloc[count]["Latitude"]
    long = hotel_df.iloc[count]["Longitude"]
    response = requests.get(f"{url}locationbias=circle%3A5000%40{lat}%2C{long}&fields=formatted_address%2Cname%2Cgeometry&key={g_key}").json()
    hotel_names.append(response["candidates"][0]["name"])
    addresses.append(response["candidates"][0]["formatted_address"])
    count += 1
    
hotel_df["Hotel Name"] = hotel_names
hotel_df.insert(1, 'Country', addresses)

In [84]:
countries = []
for address in addresses:
    country_string = address.split(", ")
    country_string = country_string[-1]
    countries.append(country_string)

hotel_df["Country"] = countries

In [86]:
hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Hotel Name
117,Choctaw,United States,35.497565,-97.268926,70.02,47,0,5.75,Best Western Plus Choctaw Inn & Suites
208,Viranşehir,Turkey,37.232895,39.762014,74.57,21,0,5.64,Tella Hotel
214,Ampanihy,Madagascar,-24.694126,44.747682,70.12,52,0,0.94,HOTEL SHARRON AMPANIHY OUEST
242,Hasilpur,Pakistan,29.688981,72.552545,76.80,71,0,9.13,The City Family Hotel
275,Turbat,Pakistan,26.002789,63.050561,78.39,11,0,7.09,Shapur Hotel
311,Tsihombe,Madagascar,-25.318547,45.484057,71.46,58,0,0.34,Littoral Hôtel
397,Cizre,Turkey,37.332437,42.185470,72.57,24,0,9.22,Divan Cizre Hotel
467,Riyadh,Saudi Arabia,24.638916,46.716010,73.78,15,0,5.48,Four Points by Sheraton Riyadh Khaldia
505,Cayenne,French Guiana,4.937114,-52.325831,77.02,88,0,3.44,Grand Hotel Montabo
557,Sehithwa,Botswana,-20.473311,22.701307,71.69,20,0,4.27,MONLEK GUEST HOUSE


In [87]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [89]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))